# Tutorial: filling in missing bands

- last verification date : 2024-10-24 (Sylvie dagoret-Campagne)
- Must run this notebook from `docs/pre-executed` folder
- NOT DEBUGGED

We will use the parameter file "tests_nb/parametersTest.cfg".
This contains a description of the bands and data to be used.
In this example we will generate mock data for the ugriz SDSS bands,
fit each object with our GP using ugi bands only and see how it predicts the rz bands.
This is an example for filling in/predicting missing bands in a fully bayesian way
with a flexible SED model quickly via our photo-z GP.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import sys,os
sys.path.append('../..')
from delight.io import *
from delight.utils import *
from delight.photoz_gp import PhotozGP

## Specifying were are the data file used for input outout

In [ ]:
# path of the config parameter file
param_path = "tests_nb"
# path where the input fluxes file are generated including the Kerenl gaussian process file generated
data_path = "data_nb"

In [ ]:
if not os.path.exists(data_path):
    os.mkdir(data_path)
if not os.path.exists(param_path):
    os.mkdir(param_path)

## Creating the parameter file
Let's create a parameter file from scratch.

In [ ]:
paramfile_txt = """
# DELIGHT parameter file
# Syntactic rules:
# - You can set parameters with : or =
# - Lines starting with # or ; will be ignored
# - Multiple values (band names, band orders, confidence levels)
#   must beb separated by spaces
# - The input files should contain numbers separated with spaces.
# - underscores mean unused column
"""

### 1) Specifying the Filters used for the photometric survey

In [ ]:
paramfile_txt += """
[Bands]
names: U_SDSS G_SDSS R_SDSS I_SDSS Z_SDSS
directory: ../../data/FILTERS
bands_fmt: res
numCoefs: 7
bands_verbose: True
bands_debug: True
bands_makeplots: False
"""

### 2) Specifying the SED templates used

In [ ]:
paramfile_txt += """
[Templates]
directory: ../../data/CWW_SEDs
sed_fmt: dat
names: El_B2004a Sbc_B2004a Scd_B2004a SB3_B2004a SB2_B2004a Im_B2004a ssp_25Myr_z008 ssp_5Myr_z008
p_t: 0.27 0.26 0.25 0.069 0.021 0.11 0.0061 0.0079
p_z_t:0.23 0.39 0.33 0.31 1.1 0.34 1.2 0.14
lambdaRef: 4.5e3
"""

### 3) Specifying the training and target photometric catalogs

In [ ]:
paramfile_txt += """
[Simulation]
numObjects: 1000
noiseLevel: 0.03
trainingFile: ./data_nb/galaxies-fluxredshifts.txt
targetFile: ./data_nb/galaxies-fluxredshifts2.txt
"""

#### 3.a Config for the simulation of the training catalog

In [ ]:
paramfile_txt += """
[Training]
catFile: ./data_nb/galaxies-fluxredshifts.txt
bandOrder: U_SDSS U_SDSS_var G_SDSS G_SDSS_var _ _ I_SDSS I_SDSS_var _ _ redshift
referenceBand: I_SDSS
extraFracFluxError: 1e-4
paramFile: ./data_nb/galaxies-gpparams.txt
crossValidate: True
CVfile: ./data_nb/galaxies-gpCV.txt
crossValidationBandOrder: _ _ _ _ R_SDSS R_SDSS_var _ _ Z_SDSS Z_SDSS_var redshift
numChunks: 1
"""

#### 3.b Config for the simulation of the target catalog

In [ ]:
paramfile_txt += """
[Target]
catFile: ./data_nb/galaxies-fluxredshifts2.txt
bandOrder: U_SDSS U_SDSS_var G_SDSS G_SDSS_var _ _ I_SDSS I_SDSS_var _ _ redshift
referenceBand: I_SDSS
extraFracFluxError: 1e-4
redshiftpdfFile: ./data_nb/galaxies-redshiftpdfs.txt
redshiftpdfFileTemp: ./data_nb/galaxies-redshiftpdfs-cww.txt
metricsFile:  ./data_nb/galaxies-redshiftmetrics.txt
metricsFileTemp:  ./data_nb/galaxies-redshiftmetrics-cww.txt
useCompression: False
Ncompress: 10
compressIndicesFile: ./data_nb/galaxies-compressionIndices.txt
compressMargLikFile: ./data_nb/galaxies-compressionMargLikes.txt
redshiftpdfFileComp: ./data_nb/galaxies-redshiftpdfs-comp.txt
"""

### 4) Specifying the hyper-parameters of the Gaussian Process fitting

In [ ]:
paramfile_txt += """
[Other]
rootDir: ./
zPriorSigma: 0.2
ellPriorSigma: 0.5
fluxLuminosityNorm: 1.0
alpha_C: 1.0e3
V_C: 0.1
alpha_L: 1.0e2
V_L: 0.1
lines_pos: 6500 5002.26 3732.22
lines_width: 20.0 20.0 20.0
redshiftMin: 0.1
redshiftMax: 1.101
redshiftNumBinsGPpred: 100
redshiftBinSize: 0.001
redshiftDisBinSize: 0.2
confidenceLevels: 0.1 0.50 0.68 0.95
"""

Let's write this to a file.

In [ ]:
with open('./tests_nb/parametersTest.cfg','w') as out:
    out.write(paramfile_txt)

In [ ]:
from delight.io import parseParamFile
params = parseParamFile('./tests_nb/parametersTest.cfg', verbose=False)

In [ ]:
params['training_CV_bandOrder']

# Running Delight

In [ ]:
# First, we must fit the band filters with a gaussian mixture. 
# This is done with this script:
%run ../../scripts/processFilters.py ./tests_nb/parametersTest.cfg

In [ ]:
# Second, we will process the library of SEDs and project them onto the filters,
# (for the mean fct of the GP) with the following script:
%run ../../scripts/processSEDs.py ./tests_nb/parametersTest.cfg

In [ ]:
# Third, we will make some mock data with those filters and SEDs:
%run ../../scripts/simulateWithSEDs.py ./tests_nb/parametersTest.cfg

In [ ]:
# Now we load the parameter file and the useful quantities
params = parseParamFile('./tests_nb/parametersTest.cfg', verbose=False)
bandCoefAmplitudes, bandCoefPositions, bandCoefWidths, norms\
    = readBandCoefficients(params)
bandNames = params['bandNames']
numBands, numCoefs = bandCoefAmplitudes.shape
redshiftDistGrid, redshiftGrid, redshiftGridGP = createGrids(params)
f_mod = readSEDs(params) # Reads the SED library.
numObjectsTraining = np.sum(1 for line in open(params['training_catFile']))
print('Number of Training Objects', numObjectsTraining)

In [ ]:
# Create the GP (create interpolation grid for kernel + other internal stuff). May take a while.
gp = PhotozGP(f_mod, bandCoefAmplitudes, bandCoefPositions, bandCoefWidths,
              params['lines_pos'], params['lines_width'],
              params['V_C'], params['V_L'],
              params['alpha_C'], params['alpha_L'],
              redshiftGridGP, use_interpolators=True)

In [ ]:
# Load useful quantities for the training set.
# "CV" means quantities for the bands used for cross-validation.
# In this example we will drop a couple of bands in the training data 
# and use them for cross-validation, as shown in the parameter file.
bandIndicesCV, bandNamesCV, bandColumnsCV,\
    bandVarColumnsCV, redshiftColumnCV =\
    readColumnPositions(params, prefix="training_CV_", refFlux=False)
numBandsCV = bandIndicesCV.size

In [ ]:
print("bandIndicesCV:",bandIndicesCV,"bandNamesCV:",bandNamesCV,"bandColumnsCV:",bandColumnsCV)
print("bandVarColumnsCV:","bandVarColumnsCV:",bandVarColumnsCV, "redshiftColumnCV:",redshiftColumnCV)

In [ ]:
# Loop and parse the training set, fit the GP to the deep bands, 
# and run cross-validation against the cross-validation bands.
# We will store a bunch of things, including the chi2 of the fit.
numZ = redshiftGrid.size
all_z = np.zeros((numObjectsTraining, ))
all_fluxes = np.zeros((numObjectsTraining, numBands))
all_fluxes_var = np.zeros((numObjectsTraining, numBands))
all_fluxesCV = np.zeros((numObjectsTraining, numBands))
all_fluxesCV_var = np.zeros((numObjectsTraining, numBands))
all_chi2s = np.zeros((numObjectsTraining, numBandsCV))
all_bestTypes = np.zeros((numObjectsTraining, ), dtype=int)

loc = - 1
trainingDataIter1 = getDataFromFile(params, 0, numObjectsTraining,
                                    prefix="training_", getXY=True,
                                    CV=True)
for z, normedRefFlux,\
    bands, fluxes, fluxesVar,\
    bandsCV, fluxesCV, fluxesVarCV,\
        X, Y, Yvar in trainingDataIter1:
    
    
    if loc<2:
        print(loc,"\t bandsCV",bandsCV)
    
    loc += 1

    # Interpolate template library at spectroscopic redshift
    themod = np.zeros((1, f_mod.shape[0], bands.size))
    for it in range(f_mod.shape[0]):
        for ib, band in enumerate(bands):
            themod[0, it, ib] = f_mod[it, band](z)
    # Run color likelihood to find best template and ML luminosity
    chi2_grid, ellMLs = scalefree_flux_likelihood(fluxes, fluxesVar, themod, returnChi2=True)
    bestType = np.argmin(chi2_grid)
    ell = ellMLs[0, bestType]
    # Use them in the GP
    X[:, 2] = ell
    gp.setData(X, Y, Yvar, bestType)
    
    # Make predictions for all bands (noiseless) from the GP
    model_mean, model_covar\
        = gp.predictAndInterpolate(np.array([z]), ell=ell)
    ind = np.array([list(bandIndicesCV).index(b) for b in bandsCV])

    print(ind)
    # Compute chi2 for SDSS bands
    all_chi2s[loc, ind] =\
        (model_mean[0, bandsCV] - fluxesCV)**2 /\
        (model_covar[0, bandsCV] + fluxesVarCV)
        
    # Store a few useful quantities
    all_z[loc] = z
    all_bestTypes[loc] = bestType
    all_fluxes[loc, bands] = fluxes
    all_fluxes_var[loc, bands] = fluxesVar
    all_fluxesCV[loc, bandsCV] = fluxesCV
    all_fluxesCV_var[loc, bandsCV] = fluxesVarCV

In [ ]:
bandsCV

In [ ]:
# Plot the chi2 distributions for each band (1 degree of freedom)
fig, axs = plt.subplots(2, 2, figsize=(6, 4), sharex=True)
x = np.linspace(0, 3, 50)
for i, ib in enumerate(bandsCV):    
    axs[0, i].plot(x, scipy.stats.chi2.pdf(x, 1), 'k')
    axs[0, i].hist(all_chi2s[:, i], 20, range=[0, x[-1]], histtype='step', normed=True, lw=1)
    axs[1, i].plot(x, scipy.stats.chi2.pdf(x, 1), 'k')
    axs[1, i].hist(all_chi2s[:, i], 20, range=[0, x[-1]], histtype='step', normed=True, lw=1)
    axs[1, i].set_yscale('log')
    axs[1, i].set_xlabel('$\chi^2$')
    axs[0, i].set_title(bandNames[ib])
    if i == 0:
        axs[0, i].set_ylabel('$F(\chi^2)$')
        axs[1, i].set_ylabel('$\log_{10}\ F(\chi^2)$')
    else:
        axs[0, i].set_yticks([])
        axs[1, i].set_yticks([])
fig.tight_layout()